In [53]:
before_period_start = '2019-03-01'
before_period_end = '2020-03-01'

after_period_start = '2020-03-01'
after_period_end = '2021-03-01'

In [54]:
import plotly.express as px

before_color = px.colors.qualitative.D3[0]
during_color = px.colors.qualitative.D3[1]
black_color = '#2D2D2D'
green_color = px.colors.qualitative.D3[2]
red_color = px.colors.qualitative.D3[3]

In [55]:
def annotate_diff(plot, x, y, offset, title, before, after):
    diff = before - after;
    diff_percent = -(diff / before) * 100
    
    plot.add_annotation(x=x, y=y+(offset * 3), text=title, showarrow=False, font=dict(
        color=black_color,
        size=18,
    ))
    plot.add_annotation(x=x - 1, y=y+(offset * 1.5), text='{:10.0f}'.format(before), showarrow=False, font=dict(
        color=before_color,
        size=14,
    ))
    plot.add_annotation(x=x - 1, y=y+offset, text='{:10.0f}'.format(after), showarrow=False, font=dict(
        color=during_color,
        size=14,
    )) 
    color = green_color if diff_percent > 0 else red_color
    xOff = -1 if diff_percent > 0 else 0
    plot.add_annotation(x=x + xOff, y=y + 20, text='{:10.1f}%'.format(diff_percent), showarrow=False, font=dict(
        color=color,
        size=16,
    ))
    #plot.add_annotation(x=x - 1, y=y, text='{:10.0f},'.format(diff), showarrow=False, font=dict(
    #    color='black',
    #    size=14,
    #))

In [56]:
def annotate_before_after(plot, show_after=True, y_placement=1425):
    before_start = parser.parse(before_period_start)
    before_end = parser.parse(before_period_end)
    before_mid = before_start + (before_end - before_start)/2
    
    after_start = parser.parse(after_period_start)
    after_end = parser.parse(after_period_end)
    after_mid = after_start + (after_end - after_start)/2
    
    plot.add_vrect(x0=before_start, x1=before_end,
                        fillcolor="#F4AC45", opacity=0.2,
                        layer="below", line_width=0)
    
    if (show_after):
        plot.add_vrect(x0=after_start, x1=after_end,
                            fillcolor="#5470AF", opacity=0.2,
                            layer="below", line_width=0)
    
    plot.add_annotation(x=before_mid, y=y_placement,
            text="Before",
            showarrow=False)
    if (show_after):
        plot.add_annotation(x=after_mid, y=y_placement,
            text="During",
            showarrow=False)

In [57]:
def annotateHourDiff(plot, steps, hour):
    before = steps['Before'][hour].sum()
    after = steps['During'][hour].sum()
    
    plot.add_annotation(x=7.5, y=300, text='{:10.0f}'.format(after - before), showarrow=False, font=dict(
        color=black_color,
        size=18,
    ))

In [58]:
def annotateMonthDiffs(plot, steps_before, steps_during):
    
    for i in range(0, 12):
        before = steps_before.steps.iloc[i]
        during = steps_during.steps.iloc[i]
        
        diff = before - during
        diff_percent = -(diff / before) * 100
        # diff_percent = -diff_percent if before >= during else diff_percent
        
        # position the label
        month = steps_during.month.iloc[i]
        offset = before - diff / 2
        
        if abs(diff) < 100:
            continue
        
        plot.add_annotation(x=month, y=offset - 275, text='{:10.0f}%'.format(diff_percent), showarrow=False, font=dict(
            color=black_color,
            size=16,
        ), xanchor="center", xshift=-14)

In [59]:
def format_title(title, subtitle=None, subtitle_font_size=14):
    title = f'<b>{title}</b>'
    if not subtitle:
        return title
    subtitle = f'<span style="font-size: {subtitle_font_size}px;">{subtitle}</span>'
    return f'{title}<br>{subtitle}'

In [60]:
def style_plot(fig, secondary_xaxis=False):
    fig.update_layout(height=500)
    fig.update_layout(xaxis=dict(tickfont=dict(size=16)), xaxis_tickformat = '%b %Y')
#    if secondary_xaxis:
        # fig.update_layout(xaxis2=dict(tickfont=dict(size=16)), xaxis2_tickformat = '%b %Y')
    fig.update_layout(
        margin=dict(l=0, r=0, t=48, b=32),
    )
    fig.update_layout(legend=dict(
        bgcolor="rgba(255,255,255, 0.0)",
        yanchor="top",
        y=0.99,
        xanchor="right",
        x=0.99
    ))
    fig.update_layout(title_font_size=24)
    fig.update_layout(font=dict(
        size=14,
        color=black_color,
        family="Lato, sans-serif"
    ))